In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json

In [2]:
response = requests.get("https://www.rightmove.co.uk/house-prices-in-Somerset.html")

In [3]:
soup = bs(response.text)

In [40]:
all_cities = soup.find_all("li", id=re.compile("sidemenu"))

In [42]:
cities = soup.find_all("li", class_=re.compile("highlightMapRegion"))

In [43]:
city_code = {}
for city in cities:
    citycode = re.search("(?<=highlightMapRegion).+", city["class"][0]).group()
    href = city.find("a")["href"]
    cityname = re.search("(?<=house-prices/).+(?=.html)", href).group()
    city_code[cityname] = [citycode, href]

In [44]:
city_code2 = {}
for city in all_cities:
    citycode = re.search("(?<=sidemenu).+", city["id"]).group()
    href = city.find("a")["href"]
    cityname = re.search("(?<=house-prices/).+(?=.html)", href).group()
    city_code2[cityname] = [citycode, href]

In [45]:
city_code_rest = {}
for c in city_code2.keys():
    if c not in city_code.keys():
        city_code_rest[c] = city_code2[c]

In [48]:
city_code_rest

{'Axbridge': ['70', 'https://www.rightmove.co.uk/house-prices/Axbridge.html'],
 'Banwell': ['100', 'https://www.rightmove.co.uk/house-prices/Banwell.html'],
 'Bath-And-North-East-Somerset': ['61243',
  'https://www.rightmove.co.uk/house-prices/Bath-And-North-East-Somerset.html'],
 'Bath-Spa': ['71093',
  'https://www.rightmove.co.uk/house-prices/Bath-Spa.html'],
 'Bathampton': ['3042',
  'https://www.rightmove.co.uk/house-prices/Bathampton.html'],
 'Batheaston': ['3044',
  'https://www.rightmove.co.uk/house-prices/Batheaston.html'],
 'Bathford': ['3046',
  'https://www.rightmove.co.uk/house-prices/Bathford.html'],
 'Beckington': ['3151',
  'https://www.rightmove.co.uk/house-prices/Beckington.html'],
 'Bishops-Lydeard': ['3640',
  'https://www.rightmove.co.uk/house-prices/Bishops-Lydeard.html'],
 'Brean': ['4458', 'https://www.rightmove.co.uk/house-prices/Brean.html'],
 'Brent-Knoll': ['4490',
  'https://www.rightmove.co.uk/house-prices/Brent-Knoll.html'],
 'Bruton': ['234', 'https://ww

In [56]:
def extract_data(div, cityname):
    dic = {}
    if div.find("a"):
        dic["city"] = cityname
        dic["building"] = div.find("a").text
        dic["soldPrice"] = div.find("td", class_="soldPrice").text
        dic["soldType"] = div.find("td", class_="soldType").text
        dic["soldDate"] = div.find("td", class_="soldDate").text
        dic["bednumber"] = div.find("td", class_="noBed").text
    return dic

In [57]:
# contents = []

# for city in city_code:
#     url = city_code[city][1]
#     citycode = city_code[city][0]
#     for index in range(0, 40*25, 25):
#         response = requests.get(url + 
#                             f"?country=england&locationIdentifier=REGION%5E{citycode}&searchLocation={city}&index={index}")
#         for d in bs(response.text).find_all("div", "soldDetails"):
#             data = extract_data(d, city)
#             if data:
#                 contents.append(data)

In [58]:
contents = []

for city in city_code:
    url = city_code[city][1]
    citycode = city_code[city][0]
    for index in range(0, 40*25, 25):
        response = requests.get(url + 
                            f"?country=england&locationIdentifier=REGION%5E{citycode}&searchLocation={city}&index={index}")
        details = bs(response.text).find_all("div", "soldDetails")
        if details:
            for d in details:
                data = extract_data(d, city)
                if data:
                    contents.append(data)
        else:
            break

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
len(contents)

In [ ]:
with open("house2.json", "w") as f:
    json.dump(contents, f)

In [219]:
# with open("house.json", "w") as f:
#     json.dump(contents, f)